In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/akbar-company/BI_Final_dataset.csv


# **Cleaning and Checking Data** 

In [7]:
# Step 1: Import libraries
import pandas as pd

# Step 2: Load the dataset
input_path = '/kaggle/input/akbar-company/BI_Final_dataset.csv'  # adjust if your folder is different
df = pd.read_csv(input_path)

# Step 3: Preview raw dataset
print("📌 Preview of Original Dataset:")
display(df.head())

# Step 4: Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
print("\n✅ Column names cleaned:")
print(df.columns.tolist())

# Step 5: Check for missing values
print("\n🔍 Missing values per column:")
missing_counts = df.isnull().sum()
display(missing_counts[missing_counts > 0])

# Step 6: Handle missing values
# You can customize this; for now:
# - drop rows with too many missing values
# - fill numeric columns with median, others with 'Unknown'
threshold = int(0.5 * df.shape[1])
df = df.dropna(thresh=threshold)

for col in df.columns:
    if df[col].dtype in ['float64', 'int64']:
        df[col] = df[col].fillna(df[col].median())
    else:
        df[col] = df[col].fillna('Unknown')

print("\n✅ Missing values handled.")

# Step 7: Handle duplicates
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f"\n✅ Duplicates removed: {before - after} rows dropped.")

# Step 8: Modify 'warehouse_location' to only have the first word
if 'warehouse_location' in df.columns:
    df['warehouse_location'] = df['warehouse_location'].astype(str).str.strip().str.split().str[0]
    print("\n✅ 'warehouse_location' modified to first word:")
    display(df['warehouse_location'].head())
else:
    print("⚠️ 'warehouse_location' column not found.")

# Step 9: Final preview
print("\n📌 Final Cleaned Dataset Preview:")
display(df.head())

# Step 10: Save cleaned dataset
output_path = '/kaggle/working/BI_Final2_dataset_cleaned.csv'
df.to_csv(output_path, index=False)
print(f"\n✅ Cleaned dataset saved to: {output_path}")


📌 Preview of Original Dataset:


,Date,Warehouse_Location,Material_ID,Material_Name,Category,Supplier_Name,Forecasted_Demand,Actual_Consumption,Safety_Stock,Reorder_Level,Current_Stock,Lead_Time_Days,Unit_Cost,Inventory_Value (PKR),Reorder_Flag
0,11/21/2024,Islamabad East,M0019,Material_089,Raw Material,Alpha Ltd,68,60,91,132,187,10,110.98,20753.26,No
1,10/15/2024,Karachi Central,M0057,Material_022,Raw Material,Gamma Traders,264,207,52,115,184,4,31.90,5869.60,No
2,12/11/2024,Karachi Central,M0069,Material_092,Finished Good,Delta Supplies,83,35,80,144,273,5,128.99,35214.27,No
3,11/30/2024,Lahore South,M0003,Material_029,Raw Material,Alpha Ltd,190,215,92,137,122,2,111.67,13623.74,Yes
4,10/21/2024,Faisalabad Hub,M0001,Material_013,Semi-Finished,Alpha Ltd,220,163,85,117,72,2,127.86,9205.92,Yes



✅ Column names cleaned:
['date', 'warehouse_location', 'material_id', 'material_name', 'category', 'supplier_name', 'forecasted_demand', 'actual_consumption', 'safety_stock', 'reorder_level', 'current_stock', 'lead_time_days', 'unit_cost', 'inventory_value_(pkr)', 'reorder_flag']

🔍 Missing values per column:


Series([], dtype: int64)


✅ Missing values handled.

✅ Duplicates removed: 0 rows dropped.

✅ 'warehouse_location' modified to first word:


0     Islamabad
1       Karachi
2       Karachi
3        Lahore
4    Faisalabad
Name: warehouse_location, dtype: object


📌 Final Cleaned Dataset Preview:


,date,warehouse_location,material_id,material_name,category,supplier_name,forecasted_demand,actual_consumption,safety_stock,reorder_level,current_stock,lead_time_days,unit_cost,inventory_value_(pkr),reorder_flag
0,11/21/2024,Islamabad,M0019,Material_089,Raw Material,Alpha Ltd,68,60,91,132,187,10,110.98,20753.26,No
1,10/15/2024,Karachi,M0057,Material_022,Raw Material,Gamma Traders,264,207,52,115,184,4,31.90,5869.60,No
2,12/11/2024,Karachi,M0069,Material_092,Finished Good,Delta Supplies,83,35,80,144,273,5,128.99,35214.27,No
3,11/30/2024,Lahore,M0003,Material_029,Raw Material,Alpha Ltd,190,215,92,137,122,2,111.67,13623.74,Yes
4,10/21/2024,Faisalabad,M0001,Material_013,Semi-Finished,Alpha Ltd,220,163,85,117,72,2,127.86,9205.92,Yes



✅ Cleaned dataset saved to: /kaggle/working/BI_Final2_dataset_cleaned.csv


^^^
Also made some changes in column warehouse_location for clarity 

# **Checking if data integration and structure is up to the mark**

In [3]:
columns_to_check = [
    'date', 'warehouse_location', 'material_id', 'material_name', 'category',
    'supplier_name', 'forecasted_demand', 'actual_consumption', 'safety_stock',
    'reorder_level', 'current_stock', 'lead_time_days', 'unit_cost',
    'inventory_value_(pkr)', 'reorder_flag'
]

print("\n📌 Unique values count for each specified column:\n")
for col in columns_to_check:
    if col in df.columns:
        print(f"{col}: {df[col].nunique()} unique values")
    else:
        print(f"⚠️ Column not found in dataset: {col}")

# Optional: Display unique values themselves (for categorical columns)
categorical_cols = ['warehouse_location', 'material_name', 'category', 'supplier_name', 'reorder_flag']
print("\n📌 Unique values for categorical columns:\n")
for col in categorical_cols:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].unique())
        print()


📌 Unique values count for each specified column:

date: 90 unique values
warehouse_location: 4 unique values
material_id: 100 unique values
material_name: 100 unique values
category: 4 unique values
supplier_name: 4 unique values
forecasted_demand: 247 unique values
actual_consumption: 298 unique values
safety_stock: 60 unique values
reorder_level: 114 unique values
current_stock: 278 unique values
lead_time_days: 13 unique values
unit_cost: 971 unique values
inventory_value_(pkr): 906 unique values
reorder_flag: 2 unique values

📌 Unique values for categorical columns:

warehouse_location:
['Islamabad East' 'Karachi Central' 'Lahore South' 'Faisalabad Hub']

material_name:
['Material_089' 'Material_022' 'Material_092' 'Material_029'
 'Material_013' 'Material_082' 'Material_002' 'Material_085'
 'Material_079' 'Material_052' 'Material_027' 'Material_050'
 'Material_095' 'Material_023' 'Material_088' 'Material_044'
 'Material_077' 'Material_043' 'Material_037' 'Material_061'
 'Material_